Import modules

In [3]:
import os
import pandas as pd 
from pathlib import Path
import string
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import io

ImportError: Missing required dependencies ['numpy']

In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

ImportError: cannot import name '_distributor_init' from 'numpy' (C:\Users\marti\Anaconda3\lib\site-packages\numpy\__init__.py)

In [ ]:
from nltk import word_tokenize, pos_tag
import re
import string
    

In [1]:
# find_files is given a directory. It creates a database containing the name and relative path for each file

def find_files(subfolder_path):
    #This code creates a database with every file and its path
    dir = Path(os.getcwd())
    #Add directory where your files are:
    newdir = dir / subfolder_path


    #subfolders = os.listdir(newdir)
    subfolders = ['Train_data', 'Test_data']
    dense_list = [os.listdir(newdir / subfolder) for subfolder in subfolders]
    paired_list = zip(dense_list, subfolders)

    audio_files = [(item, label) for sublist, label in paired_list for item in sublist]
    audio_file_list, path_list = zip(*audio_files)
    columns = ['file', 'relative_path']

    common_prefix = os.path.commonprefix([dir, newdir])
    relative_path = os.path.relpath(newdir, common_prefix)
    print(relative_path)

    #Using os.join.path and Path() leads to a Windows path, so I had to do it this way
    relative_path = [relative_path + '/' + path for path in path_list]
    print(relative_path)


    df = pd.DataFrame(columns = columns)
    print(audio_file_list)
    print(relative_path)
    df.file = audio_file_list
    df.relative_path = relative_path
    return df

section_number = 1
section_string = '_' + str(section_number) + '.'

df = find_files('Feature_representations')

non_ind = list(map(lambda x: 'test' in x or 'train' in x, list(df['file'].apply(lambda x: x.lower()))))
df = df.drop([i for i, x in enumerate(non_ind) if not x])
df.reset_index(drop = True, inplace = True)

test_ind = list(map(lambda x: 'test' in x, list(df['file'].apply(lambda x: x.lower()))))
df['test'] = df['file'][[i for i, x in enumerate(test_ind) if x]]
df['train'] = df['file'][[i for i, x in enumerate(test_ind) if not x]]
ind_list = df['file'].apply(lambda x: section_string in x).tolist()
df.drop([i for i, x in enumerate(ind_list) if not x], inplace =True)


# loadGloveData takes the dimension of the Glove word vector as input. In creates a numpy version of the word vectors that is saved to the disk. 

def loadGloveData(cur_dim):

    #%% Set path to Glove word vector folder
    dir = Path(os.getcwd())
    wvpack = "glove.6B."+str(cur_dim)+"d.txt"
    file_1 = dir / "glove.6B" / wvpack

    df = pd.read_csv(file_1, sep=" ", quoting=3, header=None, index_col=0)
    WV = {key: val.values for key, val in df.T.items()}
    file_2 = os.path.join(dir,'glove_dic','wv_dic_{}.npy'.format(cur_dim))
    np.save(file_2, WV) 

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

dim = [50, 100, 200, 300]
dir = Path(os.getcwd())
print(dir)

for example in dim:
    fname = os.path.join(dir,'glove_dic','wv_dic_{}.npy'.format(example))
    if not os.path.isfile(fname):
        print(example)
        loadGloveData(example)

# Create_train_test_w2v_matrices takes the dataframe containing the path to each train and test csv file, which is output by find_files. This function completes the processing required to create the word averaged representation of each piece of input data. The train and test sections coerce the input dataframe into the correct format, and the input is then feed to the docAveraging function. The final matrix representation is saved to the disk.

def create_path(*args):
    cur_path = os.getcwd()
    for value in args:
        cur_path  = os.path.join(cur_path, value)
    return cur_path

class Data:
    
    table = str.maketrans({key: None for key in string.punctuation})
    def __init__(self, file, rel_path):
        self.quest_num = int(re.search(r'\d+', file).group())
        if 'test' in file.lower():
            self.mat_type = 'test'
            #Test Set
            test = pd.read_csv(create_path(rel_path, file))
            column_names = test.iloc[0,:]
            test.drop([0], inplace=True)
            test.rename(columns = column_names, inplace=True)
            label_col_bound = len([type(x) for x in test.columns.values if isinstance(x, str)])
            test = test.iloc[:, :label_col_bound]
            test['labels_1'] = test[['Label-1']].apply(lambda x: ''.join(x.astype(str)),axis=1)
            test['labels_2'] = test[['Label-2']].apply(lambda x: ''.join(x.astype(str)),axis=1)
            test = test[~test['labels_1'].str.contains("e")]
            test = test[~test['labels_2'].str.contains("e")]
            self.X = test['Dialogue'].apply(lambda x : x.lower().translate(Data.table))
            self.Y = test['labels_2']
        #Train set
        elif 'train' in file.lower():
            self.mat_type = 'train'
            #Train set
            train = pd.read_csv(create_path(rel_path, file))
            train.drop([train.columns[0]], axis = 1, inplace=True)
            train.dropna(inplace=True)
            train['labels']=train.labels.apply(lambda x: ''.join([(3-len(str(x)))*'0',str(x)]))
            self.X = train['Dialogue'].apply(lambda x : x.lower().translate(Data.table))
            self.Y = train['labels']
            
def save_labels(Y, out_path):
    if not os.path.isfile(out_path):
        np.save(out_path, Y)

def get_wordnet_pos(word):
    #Map POS tag to first character lemmatize() accepts
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def Create_TFIDF_matrices(df, lemmatize = False):
    lemtz = ''
    if lemmatize == True:
        lemtz = 'lemmatized'      
    test_df = df[['test', 'relative_path']].dropna()
    train_df = df[['train', 'relative_path']].dropna()
    test_df.reset_index(inplace = True)
    train_df.reset_index(inplace = True)
    if len(test_df) <= len(train_df):
        for i in range(min([len(test_df), len(train_df)])):  
            train_data = Data(train_df.train[i], train_df.relative_path[i])
# Attributes of train_data:   mat_type, quest_num, X_train, Y_train 
            #Sanity Check
            if train_data.mat_type != 'train' or train_data.quest_num != str(i+1):
                print('Expected matrix type train, received type {}'.format(train_data.mat_type))
                print('Expected question # {}, received # {}'.format(i+1, train_data.quest_num))
                print('error')
            test_data = Data(test_df.test[i], test_df.relative_path[i])
            #Sanity Check
            if test_data.mat_type != 'test' or test_data.quest_num != str(i+1):
                print('Expected matrix type test, received type {}'.format(test_data.mat_type))
                print('Expected question # {}, received # {}'.format(i+1, test_data.quest_num))
                print('error')
            Y_train = train_data.Y
            Y_test = test_data.Y
            #Create TF-IDF Matrices (we want to tokenize the text before creating these)
            if lemmatize==True: 
                lemmatizer =WordNetLemmatizer() 
                X_test = []
                X_train = []
                for sent in test_data.X:
                    X_test.append(' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sent)])) 
                for sent in train_data.X:
                    X_train.append(' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sent)])) 
            X_test = test_data.X
            X_train = train_data.X
#             #Flatten lists
#             X_train = [item for sublist in X_train for item in sublist]
#             X_test = [item for sublist in X_test for item in sublist]
            print('Length of X_train: ' + str(len(X_train)))
            print('Length of X_test: ' + str(len(X_test)))
            for dim in range (100, 200, 100): #max 2100
                tfidf_vectorizer = TfidfVectorizer(max_features = dim)
                tfidf_matrix = tfidf_vectorizer.fit_transform(X_train).toarray()
                tfidf_matrix_Test = tfidf_vectorizer.transform(X_test).toarray()
                file_4 = create_path('tfidf_matrices','TFIDF_train_{}Question{}_{}dim.npy'.format(lemtz, train_data.quest_num, dim))
                file_5 = create_path('tfidf_matrices','TFIDF_test_{}Question{}_{}dim.npy'.format(lemtz, test_data.quest_num, dim))
                np.save(file_4, tfidf_matrix)
                np.save(file_5, tfidf_matrix_Test)
            save_labels(Y_train, create_path('labels', '{}_labels_question{}'.format(train_data.mat_type, train_data.quest_num)))
            save_labels(Y_test, create_path('labels', '{}_labels_question{}'.format(test_data.mat_type, test_data.quest_num)))


def doc2Vec(corpus, corpus_test, dim):

    trainingMatrix = np.zeros((0, dim))
    testMatrix = np.zeros((0, dim))

    doc_clean = [doc.split() for doc in corpus]
    doc_Test = [doc.split() for doc in corpus_test]

    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(doc_clean)]
    model = Doc2Vec(documents, vector_size = dim, window=4, min_count=1, workers=4)

    for i in range (0, len(doc_clean)):
        trainingMatrix = np.append(trainingMatrix, [model.infer_vector(doc_clean[i])], axis=0)
    for i in range (0, len(doc_Test)):
        testMatrix = np.append(testMatrix, [model.infer_vector(doc_Test[i])], axis=0)

    return (trainingMatrix), (testMatrix)

def feature_extraction(responses):
    featurized_responses = []
    for response in responses:
        features = []
        features.append(sum(1 for word, pos in pos_tag(word_tokenize(response)) if pos.startswith('NN')))
        features.append(sum(1 for word, pos in pos_tag(word_tokenize(response)) if pos.startswith('VB')))
        features.append(sum(1 for word, pos in pos_tag(word_tokenize(response)) if pos.startswith('MD')))
        features.append(sum(1 for word, pos in pos_tag(word_tokenize(response)) if pos.startswith('JJ')))
        features.append(sum(1 for word, pos in pos_tag(word_tokenize(response)) if pos.startswith('DT')))    
        features.append(sum(1 for word, pos in pos_tag(word_tokenize(response)) if pos.startswith('RB')))
        features.append(len(word_tokenize(response))) 
        features.append(len(re.sub("\s*","", response)))
        features.append(len([char for char in response if char in set(string.punctuation)]))
        featurized_responses.append(features)
    return features

def Create_w2v_matrices(df, data_type):
    def docAveraging(sent, WV, dim):
        summ = [0.0] * (dim)
        A = 0.0;
        sent_A = (re.sub(r"[\n(\[\])]", "", sent)).split(" ")
        for word in sent_A:
            if word in WV : #and word not in stop:
                A = A + 1.0
                summ = np.add(summ, WV[word])
        if A != 0:
            summ = summ /A
        return summ;

    if data_type == 'glove':
        dim = [50, 100, 200, 300]
    else:
        dim = [300]
    for i in range(len(df)): 
        cur_data = Data(df.file[i], df.relative_path[i])
        #Create w2v average matrices
        for wvsize in dim:
            if data_type == 'glove':
                file_2 = create_path('glove_dic','wv_dic_{}.npy'.format(wvsize))
                WV = np.load(file_2).item() 
            elif data_type == 'cc':
                file_2 = create_path('crawl-{}d-2M'.format(wvsize), 'crawl-{}d-2M.vec'.format(wvsize))
                WV = load_vectors(file_2)
            elif data_type == 'wiki':
                file_2 = create_path('wiki-news-{}d-1M'.format(dim), 'wiki-news-{}d-1M.vec'.format(wvsize))
                WV = load_vectors(file_2)
            ttMatrix = np.zeros((0, wvsize))
            print('Current word vector size: {}'.format(wvsize))
            print('Current question: {} {}'.format(cur_data.mat_type, cur_data.quest_num))
            for train_doc in cur_data.X:
                ttMatrix = np.append(ttMatrix, [np.asarray(docAveraging(train_doc, WV, wvsize))], axis=0)#.decode('utf8').strip()), WV, dim))], axis=0)
            file_3 = create_path('{}_w2v_matrices'.format(data_type),'Question{}{}_{}dimensions.npy'.format(cur_data.quest_num,cur_data.mat_type,wvsize))
            np.save(file_3, ttMatrix) 
            save_labels(cur_data.Y, create_path(df.relative_path[i], '{}labels_question{}'.format(cur_data.mat_type, cur_data.quest_num)))   

# Create_w2v_matrices(df, 'glove')
# Create_w2v_matrices(df, 'wiki')
# Create_w2v_matrices(df, 'cc')
# Create_TFIDF_matrices(df, True)
# Create_TFIDF_matrices(df, False)

ModuleNotFoundError: No module named 'gensim'